# Walk-Forward Validation

Perform walk-forward analysis to validate strategy robustness.


In [ ]:
# Configuration
strategy_name = 'spy_sma_cross'
start_date = '2020-01-01'
end_date = None
train_period = 252  # Training period in days (~1 year)
test_period = 63    # Testing period in days (~3 months)
objective = 'sharpe'


In [ ]:
# Setup
import sys
from pathlib import Path
import pandas as pd
import json

project_root = Path().absolute().parent
sys.path.insert(0, str(project_root))

from lib.validate import walk_forward
from lib.utils import get_project_root


In [ ]:
# Run walk-forward analysis
results = walk_forward(
    strategy_name=strategy_name,
    start_date=start_date,
    end_date=end_date,
    train_period=train_period,
    test_period=test_period,
    objective=objective
)

print(f"\n✓ Walk-forward analysis complete")
print(f"  Periods: {len(results['out_sample_results'])}")


In [ ]:
# Display robustness metrics
robustness = results['robustness']

print("=" * 60)
print("ROBUSTNESS METRICS")
print("=" * 60)
print(f"Walk-Forward Efficiency: {robustness.get('efficiency', 0):.3f}")
print(f"Consistency: {robustness.get('consistency', 0):.2%}")
print(f"Avg IS Sharpe: {robustness.get('avg_is_sharpe', 0):.3f}")
print(f"Avg OOS Sharpe: {robustness.get('avg_oos_sharpe', 0):.3f}")
print(f"Std OOS Sharpe: {robustness.get('std_oos_sharpe', 0):.3f}")
print(f"Number of Periods: {robustness.get('n_periods', 0)}")
print("=" * 60)


In [ ]:
# Display IS vs OOS comparison
is_df = results['in_sample_results']
oos_df = results['out_sample_results']

print("\nIn-Sample vs Out-of-Sample Comparison:")
comparison = pd.DataFrame({
    'Period': range(1, len(is_df) + 1),
    'IS Sharpe': is_df['sharpe'].values,
    'OOS Sharpe': oos_df['sharpe'].values,
})
print(comparison.to_string(index=False))
